In [32]:
!git remote add origin https://github.com/ashok-arjun/fsl_ssl

In [21]:
!ln -s /kaggle/input/caltech-birds-2011-dataset/CUB_200_2011/images ./

In [45]:
!git add .
!git commit -m "CUB runs well"

[master a5e9c89] CUB runs well
 7 files changed, 50493 insertions(+), 35 deletions(-)


In [23]:
!python write_cross_filelist.py

Traceback (most recent call last):
  File "write_cross_filelist.py", line 35, in <module>
    fname_number = [ int(re.split('_|\.', fname)[1]) for fname in fnames]
  File "write_cross_filelist.py", line 35, in <listcomp>
    fname_number = [ int(re.split('_|\.', fname)[1]) for fname in fnames]
ValueError: invalid literal for int() with base 10: 'jpg'


In [ ]:
!python train.py --dataset dogs --train_aug

wandb: Currently logged in as: multi-input (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20210528_114452-11vqc3r9
wandb: Syncing run dry-frost-7
wandb: ⭐️ View project at https://wandb.ai/meta-learners/fsl_ssl
wandb: 🚀 View run at https://wandb.ai/meta-learners/fsl_ssl/runs/11vqc3r9
wandb: Run `wandb off` to turn off syncing.

Checkpoint path: checkpoints/dogs/_resnet18_protonet_aug_5way_5shot_16query_lr0.0010
Epoch 0 | Batch 10/100 | Loss 25.791528
Epoch 0 | Batch 20/100 | Loss 19.452385
Epoch 0 | Batch 30/100 | Loss 14.727972
Epoch 0 | Batch 40/100 | Loss 11.732775
Epoch 0 | Batch 50/100 | Loss 9.849118
Epoch 0 | Batch 60/100 | Loss 8.509683
Epoch 0 | Batch 70/100 | Loss 7.547474
Epoch 0 | Batch 80/100 | Loss 6.819885
Epoch 0 | Batch 90/100 | Loss 6.253194
Epoch 0 | Batch 100/100 | Loss 5

In [9]:
ls

best_model.tar  configs.json  events.out.tfevents.1622195914.538eba3681d4
